In [1]:
#install h2o
!pip install h2o
import h2o
from h2o.automl import H2OAutoML
!pip install polars pyarrow
#initialize h2o server
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.27" 2025-04-15; OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpnny19bo6
  JVM stdout: /tmp/tmpnny19bo6/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpnny19bo6/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,3 months and 26 days
H2O_cluster_name:,H2O_from_python_unknownUser_k7alh3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.250 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [2]:
#load the data into h2o frames
data = h2o.import_file('https://raw.githubusercontent.com/dphi-official/Datasets/master/Census_Income/Training_set_census.csv')
val = h2o.import_file('https://raw.githubusercontent.com/dphi-official/Datasets/master/Census_Income/Testing_set_census.csv')

#split data into train and test datasets with 80:20 ratio
train, test = data.split_frame(ratios=[0.8])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
#remove target variable from features
x = train.columns
x.remove('income_level')

#assign target variable column name to y
y = 'income_level'

#for binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
#specify maximum number of models to try and the seed.
aml = H2OAutoML(max_models = 20, seed = 1)

#train the model
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),13/20
# GBM base models (used / total),6/7
# XGBoost base models (used / total),6/6
# DRF base models (used / total),1/2
# GLM base models (used / total),0/1
# DeepLearning base models (used / total),0/4
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [4]:
#predict dependent variable values for test dataset
y_pred = aml.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [5]:
#get automl leaderboard to view model_id and its evaluation scores. 
lb = h2o.automl.get_leaderboard(aml, extra_columns = 'ALL')
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_1_AutoML_1_20250724_65053,0.95627,0.121964,0.7248,0.180936,0.188257,0.0354408,90461,0.127287,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_1_20250724_65053,0.955234,0.123126,0.720319,0.174527,0.189037,0.0357349,54246,0.031707,StackedEnsemble
XGBoost_3_AutoML_1_20250724_65053,0.954354,0.124094,0.716139,0.179631,0.189773,0.0360136,8600,0.00297,XGBoost
XGBoost_grid_1_AutoML_1_20250724_65053_model_2,0.954246,0.124944,0.712316,0.183365,0.190683,0.0363599,7498,0.003888,XGBoost
XGBoost_grid_1_AutoML_1_20250724_65053_model_3,0.95424,0.12405,0.717163,0.182133,0.189716,0.035992,10790,0.002944,XGBoost
XGBoost_grid_1_AutoML_1_20250724_65053_model_1,0.953438,0.125754,0.707897,0.182508,0.191287,0.0365907,5115,0.00302,XGBoost
GBM_2_AutoML_1_20250724_65053,0.953024,0.126092,0.709793,0.192682,0.191069,0.0365073,15880,0.015296,GBM
XGBoost_2_AutoML_1_20250724_65053,0.953013,0.126379,0.70616,0.197867,0.191842,0.0368032,5599,0.003009,XGBoost
GBM_3_AutoML_1_20250724_65053,0.952903,0.126419,0.708515,0.185223,0.19143,0.0366453,17181,0.015524,GBM
GBM_1_AutoML_1_20250724_65053,0.952748,0.126595,0.707757,0.199407,0.191567,0.0366977,22206,0.01909,GBM


In [6]:
#predict dependent variable values for new test dataset - val
submission_pred = aml.predict(val)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.11/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'd_household_family_stat' has levels not trained on: ["Grandchild <18 ever marr not in subfamily"]
  warnings.warn(w)


In [7]:
#convert the dependent variable prediction values of new test dataset - val into pandas DataFrame
import pandas as pd
submit =pd.DataFrame({'income_level':submission_pred.as_data_frame(use_pandas=True)['predict'].tolist()})
#convert the pandas DataFrame to csv file and store it on /kaggle/working folder
submit.to_csv('/kaggle/working/sample_submission.csv',index=False)

/usr/local/lib/python3.11/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
